# Sweep and find solutions
Created 31/05/2024

Objectives:
* Sweep over and find all fermionic boundary operators.

# Package imports

In [5]:
import sys

In [6]:
sys.path.append("../../")

In [7]:
from itertools import chain
import re

from collections import Counter, namedtuple

In [8]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os
import pickle

In [9]:
import numpy as np
from jax import numpy as jnp

import matplotlib.pyplot as plt

import scipy

In [10]:
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.optimize import TNOptimizer

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:36: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:73: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


# Load data

In [11]:
DATA_DIR_1 = r"../../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/"
DATA_DIR_2 = r"../../data/interpolated_nontrivial_fermionic_proj_rep_to_nontrivial_proj_rep_200_site_dmrg/"

In [12]:
def parse_file_name(file_name):
    interpolation = int(file_name.split('_')[0])/100

    return interpolation

In [13]:
loaded_data_triv_proj_rep = dict()
energies_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_1))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_1, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_triv_proj_rep[data_info]=data['wavefunction']
        energies_triv_proj_rep[data_info]=data['energy']

In [14]:
loaded_data_non_triv_proj_rep = dict()
energies_non_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_2))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_2, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_non_triv_proj_rep[data_info]=data['wavefunction']
        energies_non_triv_proj_rep[data_info]=data['energy']

In [15]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

In [16]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_non_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

# Definitions

In [17]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [18]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [19]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [20]:
fermionic_np_symmetries = [np_I, np_JW]

In [21]:
def generate_problem_rdm(quimb_psi, symmetry_site_pairs, leftmost_symmetry_site,
                         num_symmetry_sites, num_boundary_sites):
    q_top = quimb_psi.copy(deep=True)
    for i, s in symmetry_site_pairs:
        q_top.gate(
            s,
            where=i,
            contract=False,
            inplace=True
        )

    
    indices_to_map = list(chain(
        range(leftmost_symmetry_site-num_boundary_sites, leftmost_symmetry_site),
        range(leftmost_symmetry_site+num_symmetry_sites, leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites)
    ))

    index_mapping = {f'k{i}': f'b{i}' for i in indices_to_map}

    q_bottom = (
        quimb_psi
        .copy()
        .reindex(index_mapping, inplace=True)
        .conj()
    )

    sites_to_contract = {
        'left': list(range(leftmost_symmetry_site-num_boundary_sites)),
        'middle': list(range(leftmost_symmetry_site, leftmost_symmetry_site+num_symmetry_sites)),
        'right': list(range(leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites, quimb_psi.L))
    }

    tags_to_contract = {
        k: [f'I{i}' for i in v]
        for k, v in sites_to_contract.items()
    }

    tn = (q_top & q_bottom)

    tnc = (
        tn
        .contract(tags_to_contract['left'])
        .contract(tags_to_contract['middle'])
        .contract(tags_to_contract['right'])
    )

    return tnc

In [22]:
def initialize_mpo(left_most_symmetry_site, num_symmetry_sites,
                   num_boundary_sites, bond_dimension, phys_dim=[4,2]):
    # Kind of hard coding the physical dimensions.
    ml = qtn.MPO_rand(
        num_boundary_sites,
        bond_dimension,
        phys_dim=phys_dim,
        normalize=True,
        sites=list(range(left_most_symmetry_site-num_boundary_sites, left_most_symmetry_site)),
        dtype=np.complex128,
        tags='left_mpo'
    )

    mr = qtn.MPO_rand(
        num_boundary_sites,
        bond_dimension,
        phys_dim=phys_dim,
        normalize=True,
        sites=list(range(
            left_most_symmetry_site + num_symmetry_sites,
            left_most_symmetry_site + num_symmetry_sites + num_boundary_sites
        )),
        dtype=np.complex128,
        tags='right_mpo'
    )

    mpo = (ml & mr)

    return mpo

## Optimisation functions

In [23]:
def split_mpo_pair(mpo_pair):
    ml = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo.tag_map['left_mpo']))
    )

    mr = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo.tag_map['right_mpo']))
    )

    return (ml, mr)

In [24]:
def overlap_loss_function(ml, mr, rdm_tn, epsilon=0):
    c = (rdm_tn & ml & mr) ^ ...

    c_abs_squared = (
        c
        *jnp.conjugate(c)
    )
    #c_abs_squared = c_abs_squared.astype('float32')
    c_abs = (jnp.sqrt(c_abs_squared+epsilon))

    target = jnp.sqrt(1+epsilon)
    loss = (c_abs - target)**2

    return loss

In [25]:
def overlap_loss_function_mpo_pair(mpo_pair, rdm_tn):
    ml, mr = split_mpo_pair(mpo_pair)

    return overlap_loss_function(ml, mr, rdm_tn)

In [26]:
regex_s = r"^I\d+$"
regex_p = re.compile(regex_s)

In [27]:
def relabel_mpo(mpo, k_label, b_label):
    site_locs = [
        int(k[1:]) for k in mpo.tag_map
        if bool(re.search(regex_p, k))
    ]

    k_in_indices = [f'k{i}' for i in site_locs]
    j_in_indices = [f'b{i}' for i in site_locs]

    k_out_indices = [f'{k_label}{i}' for i in site_locs]
    j_out_indices = [f'{b_label}{i}' for i in site_locs]

    mapping = dict(
        chain(
            zip(k_in_indices, k_out_indices),
            zip(j_in_indices, j_out_indices)
        )
    )

    mpo.reindex(mapping, inplace=True)

In [28]:
def unitarity_tn(tn, total_physical_dim):
    ms = [tn.copy(), tn.copy(), tn.copy()]

    relabel_mpo(ms[0], 'k', 'l')
    relabel_mpo(ms[1], 'm', 'l')
    relabel_mpo(ms[2], 'm', 'b')

    ms[0] = ms[0].conj()
    ms[2] = ms[2].conj()

    n2tn = (tn & tn.conj())
    n2 = n2tn.contract(n2tn.tag_map)
    n4tn = (tn & ms[0] & ms[1] & ms[2])
    n4 = n4tn.contract(n4tn.tag_map)

    out = jnp.real(n4 - 2*n2 + total_physical_dim)

    return out

In [29]:
def overall_loss_function(mpo_pair, rdm_tn, total_physical_dimension,
    unitary_cost_coefficient=1, overlap_cost_coefficient=1, losses=None):
    ml, mr = split_mpo_pair(mpo_pair)

    o_loss = overlap_loss_function(ml, mr, rdm_tn)
    ul_loss = unitarity_tn(ml, total_physical_dimension)
    ur_loss = unitarity_tn(mr, total_physical_dimension)

    out = (
        unitary_cost_coefficient*(ul_loss+ur_loss)
        + overlap_cost_coefficient*o_loss
    )

    out = jnp.real(out)

    if losses is not None:
        losses.append((o_loss, ul_loss, ur_loss))
    return out

# Loop

In [30]:
tenpy_mps_data = {
    'triv': loaded_data_triv_proj_rep,
    'non_triv': loaded_data_non_triv_proj_rep
}

In [31]:
num_boundary_sites=6
left_most_symmetry_site=60
num_symmetry_sites=80
bond_dimension=6

total_physical_dim = 2**9

In [32]:
num_iterations = int(1e4) 
num_seeds = 5
nhop=5

alpha=1e-4
beta_1 = 0.3
beta_2 = 0.3
overlap_learning_rate=50

score_target=0.1

In [29]:
for k1, mps_psi_dict in tenpy_mps_data.items():
    for k2, mps_psi in mps_psi_dict.items():

        psi_arrays = list()
        psi_arrays.append(mps_psi.get_B(0, 'Th')[0, ...].to_ndarray())
        for i in range(1, mps_psi.L-1):
            psi_arrays.append(mps_psi.get_B(i).to_ndarray())
        psi_arrays.append(mps_psi.get_B(mps_psi.L-1)[..., 0].to_ndarray())

        q1 = (
            qtn
            .tensor_1d
            .MatrixProductState(psi_arrays, shape='lpr')
        )

        for k3, bs in enumerate(bosonic_np_symmetries[1:], start=1):
            for k4, fs in enumerate(fermionic_np_symmetries):

                symmetry_site_pairs = (
                    [(i, bs) for i in range(left_most_symmetry_site, left_most_symmetry_site+num_symmetry_sites, 2)]
                    + [(i, fs) for i in range(left_most_symmetry_site+1, left_most_symmetry_site+num_symmetry_sites+1, 2)]
                )

                problem_rdm = generate_problem_rdm(
                    q1,
                    symmetry_site_pairs,
                    left_most_symmetry_site,
                    num_symmetry_sites,
                    num_boundary_sites
                )

                seed_number = 0
                new_seed_needed=True
                while new_seed_needed:
                    mpo = initialize_mpo(
                        left_most_symmetry_site,
                        num_symmetry_sites,
                        num_boundary_sites,
                        bond_dimension
                    )
                
                    optmzr = qtn.optimize.TNOptimizer(
                        mpo,                                
                        loss_fn=overall_loss_function,
                        loss_kwargs={
                            'rdm_tn': problem_rdm,
                            'total_physical_dimension': total_physical_dim,
                            'unitary_cost_coefficient': 1,
                            'overlap_cost_coefficient': overlap_learning_rate
                        },
                        autodiff_backend='jax',
                        optimizer='cadam',
                    )
    
                    optmzr.optimize_basinhopping(
                        num_iterations,
                        nhop,
                        learning_rate=alpha,
                        beta1=beta_1,
                        beta2=beta_2,
                        tol=score_target
                    )

                    final_score = jnp.min(np.array(optmzr.losses))

                    if (seed_number == new_seed_needed - 1) or (final_score < score_target):
                        new_seed_needed = False

                    seed_number += 1

                    file_name = rf'solutions/{k1}_{k2}_{k3}_{k4}_{seed_number}.pickle'
                    print('Saving: ' + file_name)

                    with open(file_name, 'wb') as file:
                        pickle.dump([final_score, optmzr.get_tn_opt()], file)

  0%|                                                                                                   | 0/50000 [00:00<?, ?it/s]/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/quimb/tensor/optimize.py:135: UserWarning: dtype mismatch between input parameter and updated values. This can occur e.g. with jax and double precision arrays (in which case consider setting `jax.config.update("jax_enable_x64", True)` at startup or using single precision parameters directly). For now casting from complex64 to complex128.
  warnings.warn(
+0.126451015472 [best: +0.070920400321] : : 60006it [05:14, 190.88it/s]                                                           


Saving: solutions/triv_0.65_1_0_1.pickle


+0.217400550842 [best: +0.064896956086] : : 60006it [05:15, 190.43it/s]                                                           


Saving: solutions/triv_0.65_1_1_1.pickle


+0.189543977380 [best: +0.076619632542] : : 60006it [05:14, 190.61it/s]                                                           


...

Saving: solutions/non_triv_0.5_2_1_1.pickle


+2.782383918762 [best: +2.675715446472] : : 60006it [04:02, 247.91it/s]                                                                                     


Saving: solutions/non_triv_0.5_3_0_1.pickle


+2.886706352234 [best: +1.947829723358] : : 60006it [04:01, 248.76it/s]                                                                                     


Saving: solutions/non_triv_0.5_3_1_1.pickle


## Second pass
Some of the solutions weren't good.

In [33]:
SOL_DIR = r"solutions/"

In [34]:
def parse_file_name(file_name):
    if file_name[0] == 'n':
        proj_rep=1
        b, bs, fs = file_name.split('_')[2:-1]
        b = float(b)
        bs = int(bs)
        fs = int(fs)
    elif file_name[0] == 't':
        proj_rep=0
        b, bs, fs = file_name.split('_')[1:-1]

    b = float(b)
    bs = int(bs)
    fs = int(fs)
    
    return (proj_rep, b, bs, fs)

In [35]:
boundary_operator_solutions = dict()

for local_file_name in list(os.walk(SOL_DIR))[0][2]:
    f_name = r"{}/{}".format(SOL_DIR, local_file_name, ignore_unknown=False)

    key = parse_file_name(local_file_name)

    with open(f_name, 'rb') as f:
        out = pickle.load(f)

        boundary_operator_solutions[key] = out

In [36]:
score_pairs = [
    (k, float(v[0]._value)) for k, v in boundary_operator_solutions.items()
]

In [37]:
bad_pairs = [
    p for p in score_pairs if p[1] > 1
]

In [38]:
bad_pairs

[((1, 0.25, 2, 0), 46.89072036743164),
 ((1, 0.55, 1, 0), 2.1150639057159424),
 ((1, 0.2, 2, 0), 1.8652310371398926),
 ((1, 0.5, 2, 1), 50.00383758544922),
 ((1, 0.4, 1, 1), 2.559019088745117),
 ((1, 0.5, 1, 1), 50.00250244140625),
 ((1, 0.45, 1, 1), 18.73262596130371),
 ((1, 0.05, 2, 1), 2.2860751152038574),
 ((1, 0.3, 3, 0), 7.384514808654785),
 ((1, 0.45, 3, 0), 13.148876190185547),
 ((1, 0.35, 3, 1), 49.25619888305664),
 ((1, 0.3, 3, 1), 1.6220526695251465),
 ((1, 0.1, 2, 0), 5.580609321594238),
 ((1, 0.45, 3, 1), 46.7350959777832),
 ((0, 0.5, 3, 1), 50.00543212890625),
 ((0, 0.5, 3, 0), 50.0042724609375),
 ((1, 0.55, 2, 0), 1.0456675291061401),
 ((1, 0.0, 2, 0), 1.0320236682891846),
 ((1, 0.35, 2, 0), 1.64143705368042),
 ((1, 0.5, 3, 1), 1.9478297233581543),
 ((1, 0.2, 2, 1), 3.0427615642547607),
 ((1, 0.05, 3, 1), 7.46250057220459),
 ((1, 0.35, 1, 0), 49.59267044067383),
 ((1, 0.55, 2, 1), 1.2660291194915771),
 ((1, 0.25, 2, 1), 4.749995231628418),
 ((0, 0.45, 1, 1), 1.3306088447

In [39]:
len(bad_pairs)

60

In [41]:
tenpy_mps_data

{'triv': {0.65: <tenpy.networks.mps.MPS at 0x785f4e6e1010>,
  0.45: <tenpy.networks.mps.MPS at 0x785f4e551190>,
  0.55: <tenpy.networks.mps.MPS at 0x785f4e51f6d0>,
  0.95: <tenpy.networks.mps.MPS at 0x785f4e4b0490>,
  0.3: <tenpy.networks.mps.MPS at 0x785f4e303610>,
  0.7: <tenpy.networks.mps.MPS at 0x785f4e4b2890>,
  1.0: <tenpy.networks.mps.MPS at 0x785f4e3ec650>,
  0.25: <tenpy.networks.mps.MPS at 0x785f4e356650>,
  0.2: <tenpy.networks.mps.MPS at 0x785f4e2b23d0>,
  0.75: <tenpy.networks.mps.MPS at 0x785f4e128350>,
  0.1: <tenpy.networks.mps.MPS at 0x785f4e15ec10>,
  0.9: <tenpy.networks.mps.MPS at 0x785f4e13b610>,
  0.05: <tenpy.networks.mps.MPS at 0x785f4e02b1d0>,
  0.15: <tenpy.networks.mps.MPS at 0x785f4e3ac8d0>,
  0.8: <tenpy.networks.mps.MPS at 0x785f4df282d0>,
  0.4: <tenpy.networks.mps.MPS at 0x785f4df62550>,
  0.0: <tenpy.networks.mps.MPS at 0x785f4dfda610>,
  0.6: <tenpy.networks.mps.MPS at 0x785f4de74e90>,
  0.85: <tenpy.networks.mps.MPS at 0x785f4deb8dd0>,
  0.35: <tenpy

In [ ]:
for k, _ in bad_pairs:
    k1, k2, k3, k4 = k
    k1 = 'non_triv' if k1 == 1 else 'triv'
    mps_psi = tenpy_mps_data[k1][k2]
    bs = bosonic_np_symmetries[k3]
    fs = fermionic_np_symmetries[k4]

    psi_arrays = list()
    psi_arrays.append(mps_psi.get_B(0, 'Th')[0, ...].to_ndarray())
    for i in range(1, mps_psi.L-1):
        psi_arrays.append(mps_psi.get_B(i).to_ndarray())
    psi_arrays.append(mps_psi.get_B(mps_psi.L-1)[..., 0].to_ndarray())

    q1 = (
        qtn
        .tensor_1d
        .MatrixProductState(psi_arrays, shape='lpr')
    )

    symmetry_site_pairs = (
        [(i, bs) for i in range(left_most_symmetry_site, left_most_symmetry_site+num_symmetry_sites, 2)]
        + [(i, fs) for i in range(left_most_symmetry_site+1, left_most_symmetry_site+num_symmetry_sites+1, 2)]
    )

    problem_rdm = generate_problem_rdm(
        q1,
        symmetry_site_pairs,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    seed_number = 2
    new_seed_needed=True
    while new_seed_needed:
        mpo = initialize_mpo(
            left_most_symmetry_site,
            num_symmetry_sites,
            num_boundary_sites,
            bond_dimension
        )
    
        optmzr = qtn.optimize.TNOptimizer(
            mpo,                                
            loss_fn=overall_loss_function,
            loss_kwargs={
                'rdm_tn': problem_rdm,
                'total_physical_dimension': total_physical_dim,
                'unitary_cost_coefficient': 1,
                'overlap_cost_coefficient': overlap_learning_rate
            },
            autodiff_backend='jax',
            optimizer='cadam',
        )

        optmzr.optimize_basinhopping(
            num_iterations,
            nhop,
            learning_rate=alpha,
            beta1=beta_1,
            beta2=beta_2,
            tol=score_target
        )

        final_score = jnp.min(np.array(optmzr.losses))

        if (seed_number == num_seeds - 1) or (final_score < score_target):
            new_seed_needed = False

        file_name = rf'solutions/{k1}_{k2}_{k3}_{k4}_{seed_number}.pickle'
        print('Saving: ' + file_name)

        with open(file_name, 'wb') as file:
            pickle.dump([final_score, optmzr.get_tn_opt()], file)

        seed_number += 1

+49.917625427246 [best: +49.526523590088] : : 60006it [04:46, 209.56it/s]                                                    


Saving: solutions/non_triv_0.25_2_0_2.pickle


+50.013694763184 [best: +48.802795410156] : : 60006it [04:53, 204.40it/s]                                                    


Saving: solutions/non_triv_0.25_2_0_3.pickle


+0.090379081666 [best: +0.085775628686] : : 60006it [04:57, 201.59it/s]                                                      


...

+47.948539733887 [best: +9.159541130066] : : 60006it [04:47, 209.01it/s]                                                     


Saving: solutions/non_triv_0.15_1_0_3.pickle


+10.192325592041 [best: +10.192325592041] : : 60006it [04:48, 208.24it/s]                                                    


Saving: solutions/non_triv_0.15_1_0_4.pickle


+4.139751434326 [best: +2.838873863220] :  78%|██████████████████████████████▎        | 38796/50000 [03:04<00:56, 197.11it/s]